# Parser for the BeerAdvocate websites

In this notebook, we are parsing the HTML pages that are crawled with the other notebook.

In [1]:
import os
os.chdir('..')

In [2]:
from classes.helpers import *
from classes.parser import *
import pandas as pd
import numpy as np
import datetime
import time
import gzip

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
parser = Parser()

# Parse the breweries from the places

In [5]:
parser.parse_breweries_from_places()

# Add missing breweries

In [4]:
parser.parse_missing_breweries()

# Parse the breweries page to get the number of beers

In [5]:
parser.parse_breweries_files_for_number()

# Parse the breweries page to get the beers

In [4]:
parser.parse_breweries_files_for_beers()

# Parse the beer files for information

In [43]:
parser.parse_beer_files_for_information()

# Parse the beer files for the reviews

In [4]:
df = pd.read_csv(parser.data_folder + '/parsed/beers2.csv')

In [334]:
# Open the GZIP file
f_ratings = gzip.open(parser.data_folder + 'parsed/ratings.txt.gz', 'wb')
f_reviews = gzip.open(parser.data_folder + 'parsed/reviews.txt.gz', 'wb')

weird_rat = 0
weird_rev = 0

for i in df.index:
    row = df.ix[i]
    rats = row['nbr_ratings']
    count = 0
    
    # Check that this beer has at least 1 rating
    if row['nbr_ratings'] > 0:    
        
        folder = parser.data_folder + 'beers/{}/{}/'.format(row['brewery_id'], row['beer_id'])
        
        list_ = os.listdir(folder)
        list_.sort()

        for file in list_:
            
            if folder == folder_to_watch:
                print()
                print(file)
            
            # Open the file
            html_txt = open(folder + file, 'rb').read().decode('utf-8')
            
            # Remove the \n, \r and \t characters
            html_txt = html_txt.replace('\r', '').replace('\n', '').replace('\t', '')

            # Find the ratings without the aspects
            str_ = 'alt="Photo of ([^<]*)"></a></div></div><div id="rating_fullview_content_2">'\
            '<span class="BAscore_norm">([^<]*)</span><span class="rAvg_norm">/5</span>&nbsp;&nbsp;([^|]+?)'\
            '<br><br>(.+?)<span class="muted"><a href="/community/members/(.+?)/" class="username">([^<]*)</a>, '\
            '<a href="/beer/profile/(\d+)/(\d+)/\?ba=([^#]*)\#review">(.+?)</a></span>'

            grp = re.finditer(str_, html_txt)
            
            for g in grp:
                count += 1
                # Get username and userid
                user_name = g.group(6)
                user_id = g.group(5)
                
                if user_name == '':
                    weird_rat += 1
                
                # Some user have been deleted and leave a weird trace
                if user_name != '':
                    if folder == folder_to_watch:
                        print(user_name)

                    # Get the rating
                    rating = float(g.group(2))

                    # Get the date
                    str_date = g.group(10)
                    try:
                        year = int(str_date.split(",")[1])
                        month = time.strptime(str_date[0:3], '%b').tm_mon
                        day = int(str_date.split(",")[0][4:])

                    except IndexError:
                        # Date written in a different way

                        # Get the day of the week
                        weekday = str_date.split(' at ')[0]

                        if weekday == 'Yesterday':
                            delta = 1
                        else:
                            # Transform it to number
                            day_nbr = parser.day_to_nbr[weekday]

                            # Get last time when the file was modified
                            last_modified = os.path.getmtime(folder+file) 

                            # Get the day of the week when the file was last modified
                            dt = datetime.datetime.fromtimestamp(last_modified)
                            this_day_nbr = dt.weekday()

                            # Compute difference (modulo 7 days)
                            if day_nbr > this_day_nbr:
                                delta = this_day_nbr + 7 - day_nbr
                            else:
                                delta = this_day_nbr - day_nbr

                        # Get the day when it was posted
                        day_posted = dt - datetime.timedelta(days=delta)
                        year = day_posted.year
                        month = day_posted.month
                        day = day_posted.day

                    date = int(datetime.datetime(year, month, day, 12, 0).timestamp())

                    # Check if there's some text
                    if 'characters' in g.group(5):                  
                        str_2 = '(.+?)<br>(.+?)<span class="muted">(.+?) characters</span><br><br><div>'
                        grp2 = re.search(str_2, g.group(4))

                        # Get the text
                        text = grp2.group(1).replace('<br />', '')
                        nbr_char = int(grp2.group(3).replace(',', ''))
                    else:
                        nbr_char = np.nan
                        text = np.nan

                    # Write in the file ratings.txt.gz
                    f_ratings.write('beer_name: {}\n'.format(row['beer_name']).encode('utf-8'))
                    f_ratings.write('beer_id: {:d}\n'.format(row['beer_id']).encode('utf-8'))
                    f_ratings.write('brewery_name: {}\n'.format(row['brewery_name']).encode('utf-8'))
                    f_ratings.write('brewery_id: {:d}\n'.format(row['brewery_id']).encode('utf-8'))
                    f_ratings.write('style: {}\n'.format(row['style']).encode('utf-8'))
                    f_ratings.write('abv: {}\n'.format(row['abv']).encode('utf-8'))
                    f_ratings.write('date: {:d}\n'.format(date).encode('utf-8'))
                    f_ratings.write('user_name: {}\n'.format(user_name).encode('utf-8'))
                    f_ratings.write('user_id: {}\n'.format(user_id).encode('utf-8'))
                    f_ratings.write('appearance: {}\n'.format(np.nan).encode('utf-8'))
                    f_ratings.write('aroma: {}\n'.format(np.nan).encode('utf-8'))
                    f_ratings.write('palate: {}\n'.format(np.nan).encode('utf-8'))
                    f_ratings.write('taste: {}\n'.format(np.nan).encode('utf-8'))
                    f_ratings.write('overall: {}\n'.format(np.nan).encode('utf-8'))
                    f_ratings.write('rating: {:.2f}\n'.format(rating).encode('utf-8'))
                    f_ratings.write('text: {}\n'.format(text).encode('utf-8'))
                    f_ratings.write('\n'.encode('utf-8'))

                    if nbr_char >= 150:
                        # Write in the file reviews.txt.gz
                        f_reviews.write('beer_name: {}\n'.format(row['beer_name']).encode('utf-8'))
                        f_reviews.write('beer_id: {:d}\n'.format(row['beer_id']).encode('utf-8'))
                        f_reviews.write('brewery_name: {}\n'.format(row['brewery_name']).encode('utf-8'))
                        f_reviews.write('brewery_id: {:d}\n'.format(row['brewery_id']).encode('utf-8'))
                        f_reviews.write('style: {}\n'.format(row['style']).encode('utf-8'))
                        f_reviews.write('abv: {}\n'.format(row['abv']).encode('utf-8'))
                        f_reviews.write('date: {:d}\n'.format(date).encode('utf-8'))
                        f_reviews.write('user_name: {}\n'.format(user_name).encode('utf-8'))
                        f_reviews.write('user_id: {}\n'.format(user_id).encode('utf-8'))
                        f_reviews.write('appearance: {}\n'.format(np.nan).encode('utf-8'))
                        f_reviews.write('aroma: {}\n'.format(np.nan).encode('utf-8'))
                        f_reviews.write('palate: {}\n'.format(np.nan).encode('utf-8'))
                        f_reviews.write('taste: {}\n'.format(np.nan).encode('utf-8'))
                        f_reviews.write('overall: {}\n'.format(np.nan).encode('utf-8'))
                        f_reviews.write('rating: {:.2f}\n'.format(rating).encode('utf-8'))
                        f_reviews.write('text: {}\n'.format(text).encode('utf-8'))
                        f_reviews.write('\n'.encode('utf-8'))
                
            # Find the reviews (with text and aspects)
            str_ = 'alt="Photo of ([^<]*)"></a></div></div><div id="rating_fullview_content_2">'\
            '<span class="BAscore_norm">([^<]*)</span><span class="rAvg_norm">/5</span>&nbsp;&nbsp;([^%]+?)%(?:</span>)?' \
            '<br><span class="muted">look: ([^<]*) \| smell: ([^<]*) \| taste: ([^<]*) \| feel: ([^<]*) \|  ' \
            'overall: ([^<]*)</span><br><br>(.+?)<span class="muted"><a href="/community/members/(.+?)/" ' \
            'class="username">([^<]*)</a>, <a href="/beer/profile/(\d+)/(\d+)/\?ba=([^<]*)\#review">([^<]+?)</a></span>'
            
            grp = re.finditer(str_, html_txt)
            
            for g in grp:
                count += 1
                # Username and userid
                user_id = g.group(10)
                user_name = g.group(11)
                
                if user_name == '':
                    weird_ra += 1
                
                # Some user have been deleted and leave a weird trace
                if user_name != '':                
                    # Get the ratings
                    rating = float(g.group(2))
                    appearance = float(g.group(4))
                    aroma = float(g.group(5))
                    taste = float(g.group(6))
                    palate = float(g.group(7))
                    overall = float(g.group(8))

                    # Check if there's some text
                    if 'characters' in g.group(9):                  
                        str_2 = '(.+?)<br>(.+?)<span class="muted">(.+?) characters</span><br><br><div>'
                        grp2 = re.search(str_2, g.group(9))

                        # Get the text
                        text = grp2.group(1).replace('<br />', '')
                        nbr_char = int(grp2.group(3).replace(',', ''))
                    else:
                        nbr_char = np.nan
                        text = np.nan
                    
                
                    if folder == folder_to_watch:
                        print(user_name)

                    # Date
                    str_date = g.group(15)
                    try:
                        year = int(str_date.split(",")[1])
                        month = time.strptime(str_date[0:3], '%b').tm_mon
                        day = int(str_date.split(",")[0][4:])

                    except IndexError:
                        # Date written in a different way

                        # Get the day of the week
                        weekday = str_date.split(' at ')[0]

                        if weekday == 'Yesterday':
                            delta = 1
                        else:
                            # Transform it to number
                            day_nbr = parser.day_to_nbr[weekday]

                            # Get last time when the file was modified
                            last_modified = os.path.getmtime(folder+file) 

                            # Get the day of the week when the file was last modified
                            dt = datetime.datetime.fromtimestamp(last_modified)
                            this_day_nbr = dt.weekday()

                            # Compute difference (modulo 7 days)
                            if day_nbr > this_day_nbr:
                                delta = this_day_nbr + 7 - day_nbr
                            else:
                                delta = this_day_nbr - day_nbr

                        # Get the day when it was posted
                        day_posted = dt - datetime.timedelta(days=delta)
                        year = day_posted.year
                        month = day_posted.month
                        day = day_posted.day

                    date = int(datetime.datetime(year, month, day, 12, 0).timestamp())

                    # Write in the file ratings.txt.gz
                    f_ratings.write('beer_name: {}\n'.format(row['beer_name']).encode('utf-8'))
                    f_ratings.write('beer_id: {:d}\n'.format(row['beer_id']).encode('utf-8'))
                    f_ratings.write('brewery_name: {}\n'.format(row['brewery_name']).encode('utf-8'))
                    f_ratings.write('brewery_id: {:d}\n'.format(row['brewery_id']).encode('utf-8'))
                    f_ratings.write('style: {}\n'.format(row['style']).encode('utf-8'))
                    f_ratings.write('abv: {}\n'.format(row['abv']).encode('utf-8'))
                    f_ratings.write('date: {:d}\n'.format(date).encode('utf-8'))
                    f_ratings.write('user_name: {}\n'.format(user_name).encode('utf-8'))
                    f_ratings.write('user_id: {}\n'.format(user_id).encode('utf-8'))
                    f_ratings.write('appearance: {}\n'.format(appearance).encode('utf-8'))
                    f_ratings.write('aroma: {}\n'.format(aroma).encode('utf-8'))
                    f_ratings.write('palate: {}\n'.format(palate).encode('utf-8'))
                    f_ratings.write('taste: {}\n'.format(taste).encode('utf-8'))
                    f_ratings.write('overall: {}\n'.format(overall).encode('utf-8'))
                    f_ratings.write('rating: {:.2f}\n'.format(rating).encode('utf-8'))
                    f_ratings.write('text: {}\n'.format(text).encode('utf-8'))
                    f_ratings.write('\n'.encode('utf-8'))

                    if nbr_char >= 150:
                        # Write in the file reviews.txt.gz
                        f_reviews.write('beer_name: {}\n'.format(row['beer_name']).encode('utf-8'))
                        f_reviews.write('beer_id: {:d}\n'.format(row['beer_id']).encode('utf-8'))
                        f_reviews.write('brewery_name: {}\n'.format(row['brewery_name']).encode('utf-8'))
                        f_reviews.write('brewery_id: {:d}\n'.format(row['brewery_id']).encode('utf-8'))
                        f_reviews.write('style: {}\n'.format(row['style']).encode('utf-8'))
                        f_reviews.write('abv: {}\n'.format(row['abv']).encode('utf-8'))
                        f_reviews.write('date: {:d}\n'.format(date).encode('utf-8'))
                        f_reviews.write('user_name: {}\n'.format(user_name).encode('utf-8'))
                        f_reviews.write('user_id: {}\n'.format(user_id).encode('utf-8'))
                        f_reviews.write('appearance: {}\n'.format(appearance).encode('utf-8'))
                        f_reviews.write('aroma: {}\n'.format(aroma).encode('utf-8'))
                        f_reviews.write('palate: {}\n'.format(palate).encode('utf-8'))
                        f_reviews.write('taste: {}\n'.format(taste).encode('utf-8'))
                        f_reviews.write('overall: {}\n'.format(overall).encode('utf-8'))
                        f_reviews.write('rating: {:.2f}\n'.format(rating).encode('utf-8'))
                        f_reviews.write('text: {}\n'.format(text).encode('utf-8'))
                        f_reviews.write('\n'.encode('utf-8'))
            
            if folder == folder_to_watch:
                if file == '50.html' and False:
                    asd
                    
    if count < rats:
        asd

f_ratings.close()
f_reviews.close()


0.html
Jerseyislandbeer
AlexAfonin
R_Kole
weirdohappy
Allegue04
Dentist666
JAFoley
titvs
patre_tim
lelepat
Lawleit150percent
BrickOneal
dipso123
Hyperionecta
Azpittil57
pin
Jamestregenza
igl
Spike
SmashPants
BrazilianAussie
wl0307
ChrisCage
GarthT
LeeroyAgogo

25.html
mjstacker
LathouXaris
drinkinmiles
FadetoBock
Haybeerman
Chepe
cfalovo97
SFC21
Vyronian
Molson2000
WeissBoy
RedAleMan
Wordslinger
cammo84
magpieken
puboflyons
PolarBear3
doktorhops
admiral
LiquidAmber
jdhilt
Digimaika2
Cylinsier
thehyperduck
decimator

50.html
JoeyRamone
Rlombardi75
galluch
kelvarnsen
IronDjinn
wordemupg
BDTyre
biboergosum
Shadman
Phyl21ca
andrenaline
spinrsx
shamus
Roke87
hopsolutely
AgentZero
bryehn
JohnnyBarman
Sammy
cyrenaica
biegaman
MasterSki
TheSevenDuffs
liamt07

75.html
mintjellie
peensteen
paxkey
kwjd
MrKennedy


In [332]:
count

75

In [333]:
rats

80

In [306]:
folder + file

'../data/beers/20776/69185/50.html'

In [302]:
folder_to_watch = folder

In [307]:
html_txt

'<!DOCTYPE html><html id="XenForo" lang="en-US" dir="LTR" class="Public NoJs uix_javascriptNeedsInit LoggedOut Sidebar  Responsive pageIsLtr   not_hasTabLinks  hasSearch   is-sidebarOpen hasRightSidebar is-setWidth navStyle_3 pageStyle_0 hasFlexbox" xmlns:fb="http://www.facebook.com/2008/fbml"><head><meta charset="utf-8" /><meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1" /><meta name="viewport" content="width=device-width, initial-scale=1" /><base href="https://www.beeradvocate.com/community/" /><script>var _b = document.getElementsByTagName(\'base\')[0], _bH = "https://www.beeradvocate.com/community/";if (_b && _b.href != _bH) _b.href = _bH;</script><title>Golden | The Celt Experience | Page 3 | BeerAdvocate</title><noscript><style>.JsOnly, .jsOnly { display: none !important; }</style></noscript><link rel="stylesheet" href="css.php?css=xenforo,form,public&amp;style=7&amp;dir=LTR&amp;d=1501014892" /><link rel="stylesheet" href="css.php?css=login_bar,moderator_bar,notices,u

In [330]:
str_ = 'alt="Photo of ([^<]*)"></a></div></div><div id="rating_fullview_content_2">'\
'<span class="BAscore_norm">([^<]*)</span><span class="rAvg_norm">/5</span>&nbsp;&nbsp;([^%]+?)%(?:</span>)?' \
'<br><span class="muted">look: ([^<]*) \| smell: ([^<]*) \| taste: ([^<]*) \| feel: ([^<]*) \|  ' \
'overall: ([^<]*)</span><br><br>(.+?)<span class="muted"><a href="/community/members/(.+?)/" ' \
'class="username">([^<]*)</a>, <a href="/beer/profile/(\d+)/(\d+)/\?ba=([^<]*)\#review">([^<]+?)</a></span>'

grp = re.finditer(str_, html_txt)

for g in grp:
    print(g.group(15))

Feb 12, 2012
Feb 11, 2012
Feb 06, 2012
Jan 27, 2012
Jan 22, 2012
Oct 26, 2011
Sep 25, 2011
Sep 19, 2011
Sep 13, 2011
Sep 11, 2011
Aug 01, 2011
Jul 21, 2011
Jul 19, 2011
Jul 08, 2011
Jul 03, 2011
Jun 29, 2011
Jun 29, 2011
Jun 16, 2011
Jun 09, 2011
Jun 07, 2011
Jun 05, 2011


In [97]:
folder + file

'../data/beers/297/825/0.html'

In [335]:
df['nbr_reviews'].sum()

7341

In [336]:
df['nbr_ratings'].sum()

16941

In [337]:
def parse(filename):
    """
    Parse a txt.gz file and return a generator for it

    Copyright © 2017 Gael Lederrey <gael.lederrey@epfl.ch>

    :param filename: name of the file
    :return: Generator to go through the file
    """
    file = gzip.open(filename, 'rb')
    entry = {}
    # Go through all the lines
    for line in file:
        # Transform the string-bytes into a string
        line = line.decode("utf-8").strip()

        # We check for a colon in each line
        colon_pos = line.find(":")
        if colon_pos == -1:
            # if no, we yield the entry
            yield entry
            entry = {}
            continue
        # otherwise, we add the key-value pair to the entry
        key = line[:colon_pos]
        value = line[colon_pos + 2:]
        entry[key] = value

In [339]:
grp = parse('../data/parsed/reviews.txt.gz')
i = 0
for g in grp:
    i+=1
print(i)

7221
